In [ ]:
# !apt-get update # Update apt-get repository.
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
# !wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
# !tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
# !pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

# # Set environment variables
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

# !ls

# # Initialize findspark
# import findspark
# findspark.init()

# # Create a PySpark session
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# !pip install nltk
# !pip install emot
# !pip install langdetect
# !pip install translators
# !pip install pyspark

In [1]:
# Importing necessary libraries
import numpy as np   # NumPy for numerical operations
# import pyspark.pandas as pd # Pandas for data manipulation
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk.tokenize import word_tokenize  # NLTK for natural language processing - tokenization
# nltk.download('stopwords')
from nltk.corpus import stopwords  # NLTK for stop words
from nltk.stem.lancaster import LancasterStemmer  # NLTK for stemming
from nltk.stem.wordnet import WordNetLemmatizer  # NLTK for lemmatization
from sklearn.feature_extraction.text import TfidfVectorizer  # Scikit-learn for TF-IDF vectorization
from sklearn.model_selection import train_test_split  # Scikit-learn for train-test split
from bs4 import BeautifulSoup  # BeautifulSoup for HTML parsing


# Importing emot library for emotion analysis
# !pip install emot
import emot

# Regular expression library for text processing
import re

# Language detection library
# !pip install langdetect
from langdetect import detect

# Translators library for language translation
# !pip install translators
import translators as ts

In [2]:
# # Initialize findspark
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
sc = SparkSession.builder.appName("pyspark_test").master("spark://m1:7077").\
config("spark.executor.memory","1g").\
config("spark.executor.cores", '4').\
config("spark.executor.instances", 3).\
config("spark.sql.shuffle.partitions", 32).\
config("spark.driver.memory", '1g').getOrCreate()

spark_df = sc.read.csv('hdfs://m1:8020/data/Comments.csv', header=True).limit(1000000) # Sample taken from kaggle

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/08 08:22:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# spark_df = sc.read.csv('hdfs://m1:8020/data/Comments.csv', header=True) # Sample taken from kaggle

In [5]:
spark_df.rdd.getNumPartitions()

1

In [6]:
spark_df.show(2)

+--------------+---------+----+--------------------+------------+--------------------+
|            id|course_id|rate|                date|display_name|             comment|
+--------------+---------+----+--------------------+------------+--------------------+
|      88962892|  3173036| 1.0|2021-06-29T18:54:...|       Rahul|I think a beginne...|
|Not satisfied.|     NULL|NULL|                NULL|        NULL|                NULL|
+--------------+---------+----+--------------------+------------+--------------------+
only showing top 2 rows



In [7]:
# from pyspark.sql import SparkSession
# sc = SparkSession.builder.getOrCreate()
# csv_path = "hdfs://m1:8020/data/Comments.csv"  # This will read any CSV file in the /sparkdata folder
# spark_df = sc.read.csv(csv_path, header=True)
# spark_df.show()

In [8]:
spark_df.summary().show()

24/05/08 08:23:22 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------------------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|summary|                                   id|           course_id|                rate|                date|        display_name|        comment|
+-------+-------------------------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|  count|                                 1000|                 847|                 804|                 791|                 781|            778|
|   mean|                  1.022588199136598E8|  3928900.3350515463|   4.099871134020619|                NULL|                NULL|           NULL|
| stddev|                  2.276865998921982E7|   789422.6896250268|  1.3574216484789199|                NULL|                NULL|           NULL|
|    min|                      Thank you fo...| - I am an aspiri...| CoreLocation and...|             PMO-CP"| a

In [9]:
sc.sparkContext

<SparkContext master=spark://m1:7077 appName=pyspark_test>

In [10]:
# Drop rows with null values in the 'comment' column
# spark_df=spark_df.na.drop(subset=['comment'])
from pyspark.sql.functions import col
# Filter out rows where the "comment" column is empty, contains "nan", or is null
spark_df = spark_df.filter((col("comment") != "") 
                    & (~col("comment").isNull()) 
                    & (col("comment") != "nan")
                    & (col("rate") !="")
                    & (col("rate").cast("int").isNotNull()))

# Show the resulting DataFrame
spark_df.show()

+---------+---------+----+--------------------+---------------+-------------------------------------+
|       id|course_id|rate|                date|   display_name|                              comment|
+---------+---------+----+--------------------+---------------+-------------------------------------+
| 88962892|  3173036| 1.0|2021-06-29T18:54:...|          Rahul|                 I think a beginne...|
|125535470|  4913148| 5.0|2022-10-07T11:17:...|          Marlo|                 Aviva is such a n...|
| 68767147|  3178386| 3.5|2020-10-19T06:35:...|  Yamila Andrea|                 Muy buena la intr...|
|125029758|  3175814| 5.0|2022-09-30T21:13:...|     Jacqueline|                 This course is th...|
| 76584052|  3174896| 4.5|2021-01-30T08:45:...|        Anthony|                 I found this cour...|
|124129784|  4693438| 1.0|2022-09-20T11:30:...|          Jiang|                 nothing informati...|
|121769970|  4693272| 3.5|2022-08-22T11:48:...|        Kenneth|                 Mu

In [11]:
spark_df.cache()

DataFrame[id: string, course_id: string, rate: string, date: string, display_name: string, comment: string]

In [12]:
# Count the number of duplicated rows
dup_count = spark_df.count() - spark_df.dropDuplicates().count()

# Show the count of duplicated rows
print("Number of duplicated rows:", dup_count)
spark_df = spark_df.dropDuplicates()
# Delete cache before the next step
spark_df.unpersist()
spark_df.cache()

Number of duplicated rows: 0


DataFrame[id: string, course_id: string, rate: string, date: string, display_name: string, comment: string]

In [13]:
spark_df.columns

['id', 'course_id', 'rate', 'date', 'display_name', 'comment']

In [14]:
spark_df.dtypes

[('id', 'string'),
 ('course_id', 'string'),
 ('rate', 'string'),
 ('date', 'string'),
 ('display_name', 'string'),
 ('comment', 'string')]

In [15]:
spark_df.count()

776

In [16]:

# Count the number of null values in each column
null_counts = [spark_df.where(spark_df[c].isNull()).count() for c in spark_df.columns]

# Create a dictionary to store the results
null_counts_dict = dict(zip(spark_df.columns, null_counts))

# Print the null value counts for each column
print("Null value counts:")
for column, count in null_counts_dict.items():
    print(f"{column}: {count}")

Null value counts:
id: 0
course_id: 0
rate: 0
date: 0
display_name: 0
comment: 0


In [17]:
# Remove duplicate comments by id
spark_df = spark_df.dropDuplicates(['id'])

# Drop the 'id', 'display_name', and 'course_id' columns
columns_to_drop = [ 'display_name', 'course_id','date']
spark_df = spark_df.drop(*columns_to_drop)
# Delete cache before the next step
spark_df.unpersist()
spark_df.cache()

DataFrame[id: string, rate: string, comment: string]

In [18]:
spark_df.show()

+---------+----+-------------------------------------+
|       id|rate|                              comment|
+---------+----+-------------------------------------+
|100041608| 1.0|                 Not sure if the a...|
|100395040| 5.0|                 I would love to t...|
|100510019| 4.0|                 Un petit peu plus...|
|100647144| 5.0|                                Nett!|
|100779534| 3.5|                 PURTROPPO NELLA R...|
|100840994| 5.0|小太郎シリーズ３コース受講しました...|
|100898808| 5.0|                 Olá professor, vo...|
|100912186| 4.0|                 The nuances may h...|
|101010552| 5.0|                 A great addition ...|
|101071688| 1.0|                 A lot of the answ...|
|101109204| 4.5|                            Praktisch|
|101203926| 5.0|                 Effettivamente mi...|
|101204946| 5.0|                 Luca mi hai apert...|
|101205468| 5.0|                 Finalmente un cor...|
|101221274| 2.5|                 very basic andnot...|
|101365700| 1.5|           

In [19]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import BooleanType
from langdetect import detect

In [20]:
# Function to determine if a comment contains only numbers, blanks, or special characters
def contains_only_numbers_or_special_chars(comment):
    # Remove special characters and spaces
    comment=str(comment)
    cleaned_text = re.sub(r'\W+', '',comment)
    # Remove special numbers
    cleaned_text =re.sub(r'\d+', '', cleaned_text)
    return cleaned_text.strip()==''
contains_only_numbers_or_special_chars_udf = udf(contains_only_numbers_or_special_chars, BooleanType())

In [21]:
# Function to detect language (similar to the previous example)
def detect_language(comment):
    try:
        return detect(str(comment))
    except:
        return 'undetermined'  # Handle cases where language detection fails
detect_language_udf = udf(detect_language, StringType())

In [22]:
spark_df = spark_df.withColumn('detected_language', detect_language_udf(spark_df['comment']))
# Delete cache before the next step
spark_df.unpersist()
spark_df.cache()

DataFrame[id: string, rate: string, comment: string, detected_language: string]

In [23]:
spark_df.show()

+---------+----+-------------------------------------+-----------------+
|       id|rate|                              comment|detected_language|
+---------+----+-------------------------------------+-----------------+
|100041608| 1.0|                 Not sure if the a...|               en|
|100395040| 5.0|                 I would love to t...|               en|
|100510019| 4.0|                 Un petit peu plus...|               fr|
|100647144| 5.0|                                Nett!|               no|
|100779534| 3.5|                 PURTROPPO NELLA R...|               pt|
|100840994| 5.0|小太郎シリーズ３コース受講しました...|               ja|
|100898808| 5.0|                 Olá professor, vo...|               pt|
|100912186| 4.0|                 The nuances may h...|               en|
|101010552| 5.0|                 A great addition ...|               en|
|101071688| 1.0|                 A lot of the answ...|               en|
|101109204| 4.5|                            Praktisch|              

In [24]:
from pyspark.sql.functions import col
spark_df = spark_df.filter(~contains_only_numbers_or_special_chars_udf(spark_df["comment"]))
# Delete cache before the next step
spark_df.unpersist()
spark_df.cache()

DataFrame[id: string, rate: string, comment: string, detected_language: string]

In [25]:
spark_df.show()

+---------+----+-------------------------------------+-----------------+
|       id|rate|                              comment|detected_language|
+---------+----+-------------------------------------+-----------------+
|100041608| 1.0|                 Not sure if the a...|               en|
|100395040| 5.0|                 I would love to t...|               en|
|100510019| 4.0|                 Un petit peu plus...|               fr|
|100647144| 5.0|                                Nett!|               no|
|100779534| 3.5|                 PURTROPPO NELLA R...|               pt|
|100840994| 5.0|小太郎シリーズ３コース受講しました...|               ja|
|100898808| 5.0|                 Olá professor, vo...|               pt|
|100912186| 4.0|                 The nuances may h...|               en|
|101010552| 5.0|                 A great addition ...|               en|
|101071688| 1.0|                 A lot of the answ...|               en|
|101109204| 4.5|                            Praktisch|              

In [26]:
# how many language are not recognized
spark_df.filter(spark_df.detected_language == "undetermined").count()

0

In [27]:
#  do not transalte english comments
# non_english_comments=df[(df['detected_language'] != 'en')]
non_english_comments=spark_df.filter(spark_df.detected_language != "en")
# Delete cache before the next step
non_english_comments.unpersist()
non_english_comments.cache()

DataFrame[id: string, rate: string, comment: string, detected_language: string]

In [28]:

# Function to translate comments
def translate_comment(comment):
    try:
        translated_text = ts.translate_text(comment)
    except Exception:
        translated_text = "Error in translation: "
    return translated_text

# Register UDF
translate_comment_udf = udf(translate_comment, StringType())

# Add new column 'cleaned_comment' to the DataFrame
non_english_comments = non_english_comments.withColumn('cleaned_comment', translate_comment_udf(spark_df['comment']))
# Delete cache before the next step
non_english_comments.unpersist()
non_english_comments.cache()

DataFrame[id: string, rate: string, comment: string, detected_language: string, cleaned_comment: string]

In [29]:
non_english_comments.show()

+---------+----+-------------------------------------+-----------------+--------------------+
|       id|rate|                              comment|detected_language|     cleaned_comment|
+---------+----+-------------------------------------+-----------------+--------------------+
|100510019| 4.0|                 Un petit peu plus...|               fr|A little more ske...|
|100647144| 5.0|                                Nett!|               no|               Nice!|
|100779534| 3.5|                 PURTROPPO NELLA R...|               pt|UNFORTUNATELY IN ...|
|100840994| 5.0|小太郎シリーズ３コース受講しました...|               ja|I took 3 courses ...|
|100898808| 5.0|                 Olá professor, vo...|               pt|Hello professor, ...|
|101109204| 4.5|                            Praktisch|               de|           Practical|
|101203926| 5.0|                 Effettivamente mi...|               it|Actually, you hav...|
|101204946| 5.0|                 Luca mi hai apert...|               it|Luca,

In [30]:
spark_df=spark_df.withColumn("cleaned_comment", spark_df["comment"])
# Delete cache before the next step
spark_df.unpersist()
spark_df.cache()

DataFrame[id: string, rate: string, comment: string, detected_language: string, cleaned_comment: string]

In [31]:
spark_df=non_english_comments.union(spark_df).dropDuplicates(['id'])
# Delete cache before the next step
spark_df.unpersist()
spark_df.cache()

DataFrame[id: string, rate: string, comment: string, detected_language: string, cleaned_comment: string]

In [32]:
spark_df.show()

+---------+----+-------------------------------------+-----------------+--------------------+
|       id|rate|                              comment|detected_language|     cleaned_comment|
+---------+----+-------------------------------------+-----------------+--------------------+
|101407606| 4.0|                 it would have bee...|               en|it would have bee...|
|104751696| 5.0|                       Sí, muy buena.|               es|     Yes, very good.|
|107166928| 5.0|                 Excellent beginne...|               en|Excellent beginne...|
|108180506| 4.5|                 Hasta este moment...|               es|So far it's good ...|
|115959558| 5.0|                      Amazing company|               tl|     Amazing company|
|116593840| 5.0|自分の強みを見つける方法を解説して...|               ja|He explains how t...|
|118087848| 1.5|                 kurs jest za szyb...|               pl|The course is bei...|
|120121662| 3.0|                 J'aime bien ce fo...|               fr|I lik

In [34]:
# spark_df.write.csv("hdfs://m1:8020/output2.csv")

In [35]:
from pyspark.sql.functions import when
# Set sentiment based on rate
spark_df = spark_df.withColumn('sentiment',
                   when(spark_df['rate'] < 2, 'negative')
                   .when((spark_df['rate'] >= 2) & (spark_df['rate'] <= 3), 'neutral')
                   .otherwise('positive'))
spark_df = spark_df.withColumn('label',
                   when(spark_df['rate'] < 2, 0)
                   .when((spark_df['rate'] >= 2) & (spark_df['rate'] <= 3), 50)
                   .otherwise(99))


In [36]:
spark_df.groupBy("sentiment").count().show()

+---------+-----+
|sentiment|count|
+---------+-----+
|  neutral|  110|
| positive|  574|
| negative|   90|
+---------+-----+



In [37]:
#  convert emoji and emoticon into words
emot_obj = emot.core.emot()
def replace_emojis_with_meanings(text):
    emojis = emot_obj.emoji(text)['value']
    emojis_meanings =  emot_obj.emoji(text)['mean']
    emoticons = emot_obj.emoticons(text)['value']
    emoticons_meanings =  emot_obj.emoticons(text)['mean']

    # Remove special characters from meanings list
    emojis_meanings = [re.sub(r'[^\w\s]', '', meaning) for meaning in emojis_meanings]
     # Remove special characters from meanings list
    emoticons_meanings = [re.sub(r'[^\w\s]', '', meaning) for meaning in emoticons_meanings]

    # Replace emojis in the text with the corresponding meanings
    for emoji, meaning in zip(emojis, emojis_meanings):
        text = text.replace(emoji, meaning)

     # Replace emojis in the text with the corresponding meanings
    for emoji, meaning in zip(emoticons, emoticons_meanings):
        text = text.replace(emoji, meaning)

    return text
replace_emojis_with_meanings_udf = udf(replace_emojis_with_meanings, StringType())

In [38]:
# Function to remove links using regular expressions
def remove_links(comment):
    # Regular expression pattern to match URLs
    pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(pattern, '', comment)
remove_links_udf = udf(remove_links, StringType())

In [39]:
# Function to remove HTML tags
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    # Get the text without HTML tags
    text_without_tags = soup.get_text(separator=" ", strip=True)
    return text_without_tags
remove_html_tags_udf = udf(remove_html_tags, StringType())

In [40]:
# Function to remove_numbers_or_special_chars
def remove_numbers_or_special_chars(comment):
     # Replace escape sequences with an empty string
    cleaned_text = re.sub(r'\\[^\s]', ' ', comment)
    # convert n't into not EX(didn't => did not)
    cleaned_text= cleaned_text.replace("n't", " not")
    cleaned_text= cleaned_text.replace("_", " ")
    # Remove special characters and spaces
    cleaned_text = re.sub(r'\W+', ' ',cleaned_text)
    # Remove special numbers
    cleaned_text =re.sub(r'\d+', ' ', cleaned_text)
    return cleaned_text
remove_numbers_or_special_chars_udf = udf(remove_html_tags, StringType())

In [41]:
# apply cleaning functions on comments
def clean_text(comment):
    text_res = replace_emojis_with_meanings(comment)
    text_res = text_res.lower()
    text_res = remove_links(text_res)
    text_res = remove_html_tags(text_res)
    text_res = remove_numbers_or_special_chars(text_res)
    return text_res
clean_text_udf = udf(clean_text, StringType())

In [42]:
spark_df.show()

+---------+----+-------------------------------------+-----------------+--------------------+---------+-----+
|       id|rate|                              comment|detected_language|     cleaned_comment|sentiment|label|
+---------+----+-------------------------------------+-----------------+--------------------+---------+-----+
|101407606| 4.0|                 it would have bee...|               en|it would have bee...| positive|   99|
|104751696| 5.0|                       Sí, muy buena.|               es|     Yes, very good.| positive|   99|
|107166928| 5.0|                 Excellent beginne...|               en|Excellent beginne...| positive|   99|
|108180506| 4.5|                 Hasta este moment...|               es|So far it's good ...| positive|   99|
|115959558| 5.0|                      Amazing company|               tl|     Amazing company| positive|   99|
|116593840| 5.0|自分の強みを見つける方法を解説して...|               ja|He explains how t...| positive|   99|
|118087848| 1.5|           

In [43]:
# Apply the clean_text function to the 'cleaned_comment' column
spark_df = spark_df.withColumn('cleaned_comment', clean_text_udf(spark_df['cleaned_comment']))

In [44]:
# remove stop words and lemmatization

stop_words = set(stopwords.words('english'))
# Words to keep

words_to_keep = {'very','too','so','not','no','but'}
# keep words like "very", and "so"
stop_words = {word for word in stop_words if  word not in words_to_keep}

lemmatizer = WordNetLemmatizer()

def preprocess_text(comment):
      tokens = word_tokenize(comment.lower())  # Tokenization
      tokens = [token for token in tokens if token.isalpha()]  # Remove non-alphabetic tokens
      tokens = [token for token in tokens if token not in stop_words]  # Remove stop words
      tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatization
      return ' '.join(tokens)
preprocess_text_udf = udf(preprocess_text, StringType())

In [45]:
spark_df = spark_df.withColumn('cleaned_text', preprocess_text_udf(spark_df['cleaned_comment']))

In [46]:
spark_df.show()

+---------+----+-------------------------------------+-----------------+--------------------+---------+-----+--------------------+
|       id|rate|                              comment|detected_language|     cleaned_comment|sentiment|label|        cleaned_text|
+---------+----+-------------------------------------+-----------------+--------------------+---------+-----+--------------------+
|101407606| 4.0|                 it would have bee...|               en|it would have bee...| positive|   99|would lot better ...|
|104751696| 5.0|                       Sí, muy buena.|               es|      yes very good | positive|   99|       yes very good|
|107166928| 5.0|                 Excellent beginne...|               en|excellent beginne...| positive|   99|excellent beginne...|
|108180506| 4.5|                 Hasta este moment...|               es|so far it s good ...| positive|   99|so far good also ...|
|115959558| 5.0|                      Amazing company|               tl|     amazin

In [67]:
spark_df.columns

['id',
 'rate',
 'comment',
 'detected_language',
 'cleaned_comment',
 'sentiment',
 'label',
 'cleaned_text']

In [1]:
from couchdb import Server


# Create a CouchDB Server object
server = Server("http://172.26.0.5:5984/")

# Authenticate with CouchDB (if required)
server.resource.credentials = ("admin", "admin")

# Create or get the database
db_name = "nlp_db"
db = server.create(db_name)

# Loop over each row in the DataFrame and save to CouchDB
for row in spark_df.collect():
    doc = {
        "_id": row.id,
        "rate": row.rate,
        "comment": row.comment,
        "detected_language": row.detected_language,
        "cleaned_comment": row.cleaned_comment,
        "sentiment": row.sentiment,
        "label": row.label,
        "cleaned_text": row.cleaned_text
    }
    db.save_doc(doc)

In [47]:
# # Define a dictionary to map sentiment values to label values
# sentiment_label_map = {"positive": 1, "neutral": 0, "negative": -1}

# # Loop over sentiment values and set new column called 'label'
# for sentiment, label in sentiment_label_map.items():
#     spark_df = spark_df.withColumn("label", when(spark_df["sentiment"] == sentiment, label))
# spark_df.show(spark_df.count())

In [48]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
# Sample data
# Replace `spark_df` with your actual Spark DataFrame containing 'cleaned_text' and 'sentiment' columns
# Assuming 'sentiment' is your target column and 'cleaned_text' is your feature column
# Replace `test_ratio` with your desired test set ratio
test_ratio = 0.2

# Split the data into train and test sets
train_data, test_data = spark_df.randomSplit([1 - test_ratio, test_ratio], seed=42)

# Define the stages of the pipeline
tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="words")
count_vectorizer = CountVectorizer(inputCol="words", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")
# Define the classifier model
rf = RandomForestClassifier(featuresCol="features", labelCol="label", seed=42)

# Define the pipeline
pipeline = Pipeline(stages=[ tokenizer,count_vectorizer, idf, rf])

# Train the model
model = pipeline.fit(train_data)





In [49]:
# Make predictions on the test data
predictions = model.transform(test_data)

# Show some sample predictions
predictions.select("cleaned_text", "sentiment", "prediction").show()


+--------------------+---------+----------+
|        cleaned_text|sentiment|prediction|
+--------------------+---------+----------+
|excellent beginne...| positive|      99.0|
|course run too fa...| negative|      99.0|
| nothing information| negative|      99.0|
|                 bad| negative|      99.0|
|learn thing seeme...| positive|      99.0|
|great understandi...| positive|      99.0|
|took course kotar...| positive|      99.0|
|gone question so ...| negative|      99.0|
|very least provid...|  neutral|      99.0|
|                good| positive|      99.0|
|very disappointed...| negative|      99.0|
|due language acce...|  neutral|      99.0|
|finally valid cou...| positive|      99.0|
|            anything| negative|      99.0|
|very practical us...| positive|      99.0|
|definitely course...| positive|      99.0|
|good knowledgable...| positive|      99.0|
|           no so far| negative|      99.0|
|found unclear exp...| negative|      99.0|
|        great course| positive|

In [50]:
# prompt: with my model make prodection for this comment "i really hate that !"

comment = "i really love that !"
cleaned_comment = clean_text(comment)
cleaned_text = preprocess_text(cleaned_comment)
data = sc.createDataFrame([(cleaned_text, )], ["cleaned_text"])
predictions = model.transform(data)
predicted_sentiment = predictions.select("prediction").collect()[0][0]
print(f"Predicted sentiment: {predicted_sentiment}")


Predicted sentiment: 99.0


In [51]:
# # Evaluate the model (example: using MulticlassClassificationEvaluator)
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
# accuracy = evaluator.evaluate(predictions)
# print("Test Accuracy =", accuracy)

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.pipeline import Pipeline

## Load the data into a Spark DataFrame
df = spark_df[["cleaned_text", "label"]]

## Split the data into training and testing sets
(trainDF, testDF) = df.randomSplit([0.8, 0.2], seed=42)

## Define a pipeline with Tokenizer, HashingTF, IDF, and Logistic Regression
tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="features", numFeatures=5000)
idf = IDF(inputCol="features", outputCol="tfidf_features")
lr = LogisticRegression(featuresCol="tfidf_features", labelCol="label")

pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lr])

## Train the model
model = pipeline.fit(trainDF)

## Make predictions
trainPredictions = model.transform(trainDF)
testPredictions = model.transform(testDF)

## Calculate accuracy
trainAccuracy = MulticlassClassificationEvaluator(labelCol="lable", predictionCol="prediction", metricName="accuracy").evaluate(trainPredictions)
testAccuracy = MulticlassClassificationEvaluator(labelCol="lable", predictionCol="prediction", metricName="accuracy").evaluate(testPredictions)

print(f"Training Accuracy: {trainAccuracy:.4f}")
print(f"Testing Accuracy: {testAccuracy:.4f}")


In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.pipeline import Pipeline

## Load the data into a Spark DataFrame


## Define a pipeline with Tokenizer, HashingTF, IDF, and Multinomial Naive Bayes
tokenizer = Tokenizer(inputCol="cleaned_text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="features", numFeatures=5000)
idf = IDF(inputCol="features", outputCol="tfidf_features")
nb = NaiveBayes(featuresCol="tfidf_features", labelCol="label")

pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, nb])

## Train the model
model = pipeline.fit(trainDF)

## Make predictions
trainPredictions = model.transform(trainDF)
testPredictions = model.transform(testDF)

## Calculate accuracy
trainAccuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy").evaluate(trainPredictions)
testAccuracy = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy").evaluate(testPredictions)

print(f"Multinomial Naive Bayes Training Accuracy: {trainAccuracy:.4f}")
print(f"Multinomial Naive Bayes Testing Accuracy: {testAccuracy:.4f}")


In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import SVM
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.pipeline import Pipeline


## Define a pipeline with Tokenizer, HashingTF, IDF, and Support Vector Machine
tokenizer = Tokenizer(inputCol="clean_text", outputCol="words")
hashingTF = HashingTF(inputCol="words", outputCol="features", numFeatures=5000)
idf = IDF(inputCol="features", outputCol="tfidf_features")
svm = SVM(featuresCol="tfidf_features", labelCol="sentiment", kernel="linear", C=1.0)

pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, svm])

## Train the model
model = pipeline.fit(trainDF)

## Make predictions
trainPredictions = model.transform(trainDF)
testPredictions = model.transform(testDF)

## Calculate accuracy
trainAccuracy = MulticlassClassificationEvaluator(labelCol="sentiment", predictionCol="prediction", metricName="accuracy").evaluate(trainPredictions)
testAccuracy = MulticlassClassificationEvaluator(labelCol="sentiment", predictionCol="prediction", metricName="accuracy").evaluate(testPredictions)

print(f"Accuracy: {testAccuracy:.2f}")

## Generate classification report
report = MulticlassClassificationEvaluator(labelCol="sentiment", predictionCol="prediction", metricName="f1").evaluate(testPredictions)
print("Classification Report:")
print(f"F1 Score: {report:.2f}")
